# Iceberg Lab 
## Unit 3: CRUD Support

This lab is powered by Dataproc Serverless Spark.

In the previous unit, we did the following-
1. Create Partitioned and Unpartitioned Icberg Table in Hive Catalog
2. Explore folder structure in Hive Catalog
3. Explore table descriptions and metadata files for tables in both catalogs


In this unit, we will learn how to -
1. Delete a record and study the metadata log
2. Insert a record and study the metadata log
3. Update a record and study the metadata log
4. Insert Overwrite from a source table and study the metadata log
5. Merge from a source table and study the metadata log 

### 1. Imports

In [1]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark.sparkContext.setLogLevel("WARN")
spark

24/05/13 16:42:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-diy-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-diy-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  11002190840


In [6]:
DPMS_NAME=f"dll-hms-{PROJECT_NUMBER}"
LOCATION="us-central1"

metastore_dir = !gcloud metastore services describe $DPMS_NAME --location $LOCATION |grep 'hive.metastore.warehouse.dir'| cut -d':' -f2- | xargs 
HIVE_METASTORE_WAREHOUSE_DIR = metastore_dir[0]

print("HIVE_METASTORE_WAREHOUSE_DIR",HIVE_METASTORE_WAREHOUSE_DIR)

HIVE_METASTORE_WAREHOUSE_DIR gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse


In [7]:
TABLE_NAME="loans_by_state_iceberg"
DB_NAME="loan_db"
#fully qualified table name
FQTN=f"{DB_NAME}.{TABLE_NAME}"
print("Fully quailified table name :",FQTN)

Fully quailified table name : loan_db.loans_by_state_iceberg


In [8]:
# Checking the initial folder structure

!gsutil ls -r {HIVE_METASTORE_WAREHOUSE_DIR}/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-525c32e9-c437-4d4c-a7a0-c6fac91bba85-0-00001.parquet

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00000-ff4fdfd8-1029-48c2-ac68-d44d499bc9ce.metadata.json
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/9702db53-a27d-47ad-ab43-477bbf42516d-m0.avro
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/

### 4. DELETE operation

In [9]:
spark.sql(f"SELECT * FROM {FQTN}").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+----------+----------+
|addr_state|loan_count|
+----------+----------+
|AZ        |10318     |
|SC        |5460      |
|LA        |5284      |
|MN        |8031      |
|NJ        |16367     |
|DC        |1059      |
|OR        |5258      |
|VA        |12775     |
|RI        |1968      |
|KY        |4287      |
|WY        |964       |
|NH        |2148      |
|MI        |11638     |
|NV        |6309      |
|WI        |5798      |
|ID        |522       |
|CA        |62090     |
|CT        |6767      |
|NE        |1299      |
|MT        |1220      |
+----------+----------+
only showing top 20 rows



In [10]:
spark.sql(f"DELETE FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

++
||
++
++



In [11]:
spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
+----------+----------+



In [12]:
# Explore the table folder contents after DELETE Operation
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-525c32e9-c437-4d4c-a7a0-c6fac91bba85-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00098-3-ba66277d-68ee-4b82-86ac-0b7e1aab54f5-0-00001.parquet

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00000-ff4fdfd8-1029-48c2-ac68-d44d499bc9ce.metadata.json
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hiv

**NOTE:** 
1. In Data folder, a new  parquet file is added which has all records of addr_state="AZ" removed
2. In Metadata folder, we have new metadata, manifest lists and manifest files 

In [13]:
#Fetching the latest metadata file

#Note: In production usage the latest version of Metadata file is not always the one with highest sequence number. As we see further in lab Time Travel can restore an earlier version of metadata as current.

latest_metadata = !gsutil ls  $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/metadata/*.metadata.json | tail -1
LATEST_METADATA_FILE = latest_metadata[0]
print("LATEST_METADATA_FILE", LATEST_METADATA_FILE)

LATEST_METADATA_FILE gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00001-45382348-1735-41f3-9369-c570189ed61c.metadata.json


In [14]:
#Explore the contents of the latest metadata file

#Note that the new metadata files keeps track of new snapshots and also indicates the no of files deleted or modified 

!gsutil cat {LATEST_METADATA_FILE}

{
  "format-version" : 2,
  "table-uuid" : "9228ae9b-8c1d-4c94-bc1e-9205b86ffd30",
  "location" : "gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg",
  "last-sequence-number" : 2,
  "last-updated-ms" : 1715618925421,
  "last-column-id" : 2,
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  } ],
  "default-spec-id" : 0,
  "partition-specs" : [ {
    "spec-id" : 0,
    "fields" : [ ]
  } ],
  "last-partition-id" : 999,
  "default-sort-order-id" : 0,
  "sort-orders" : [ {
    "order-id" : 0,
    "fields" : [ ]
  } ],
  "properties" : {
    "owner" : "spark",
    "write.parquet.compression-codec" : "zstd"
  },
  "current-snapshot-id" : 5233765456160638845,
  "re

### 5. INSERT Operation


In [15]:
spark.sql(f"INSERT INTO {FQTN} VALUES ('AZ',50000)")

DataFrame[]

In [16]:
spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|AZ        |50000     |
+----------+----------+



In [17]:
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-525c32e9-c437-4d4c-a7a0-c6fac91bba85-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-204-10f4d0c1-80ac-4e51-acf9-b918ef057a8d-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00098-3-ba66277d-68ee-4b82-86ac-0b7e1aab54f5-0-00001.parquet

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/h

**NOTE:** INSERT operation has created new set of files in the table directory
1. New parquet file in data folder with new row for "AZ" 
2. New snapshot, manifest, meta-data files in the meta-data folder

In [18]:
#Fetching the latest metadata file

latest_metadata = !gsutil ls  $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/metadata/*.metadata.json | tail -1
LATEST_METADATA_FILE = latest_metadata[0]
print("LATEST_METADATA_FILE", LATEST_METADATA_FILE)

LATEST_METADATA_FILE gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00002-7ff9d324-2678-42c0-8cf0-10e0efd94222.metadata.json


In [19]:
#Explore contents of latest metadata file post INSERT operation

!gsutil cat {LATEST_METADATA_FILE}

{
  "format-version" : 2,
  "table-uuid" : "9228ae9b-8c1d-4c94-bc1e-9205b86ffd30",
  "location" : "gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg",
  "last-sequence-number" : 3,
  "last-updated-ms" : 1715619019235,
  "last-column-id" : 2,
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  } ],
  "default-spec-id" : 0,
  "partition-specs" : [ {
    "spec-id" : 0,
    "fields" : [ ]
  } ],
  "last-partition-id" : 999,
  "default-sort-order-id" : 0,
  "sort-orders" : [ {
    "order-id" : 0,
    "fields" : [ ]
  } ],
  "properties" : {
    "owner" : "spark",
    "write.parquet.compression-codec" : "zstd"
  },
  "current-snapshot-id" : 7272046516454809414,
  "re

**NOTE:**
1. The value of current snapshot is set to uuid of the latest snapshot file
2. The operation is indicated as "append" and 
3. Note the number of files added/deleted as part of this INSERT operation. 
    
    
   In our specific case since we are using unpartitioned table here all 51 records are part of the same file hence the entire file is being overwriten. 


### 6. UPDATE Operation

In [20]:
spark.sql(f"UPDATE {FQTN} SET loan_count = 11111 WHERE addr_state='AZ'").show(truncate=False)

++
||
++
++



In [21]:
spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state='AZ'").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|AZ        |11111     |
+----------+----------+



In [22]:
# Note how the update created a new parquet file and added more metadata files in the metadata directory
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-525c32e9-c437-4d4c-a7a0-c6fac91bba85-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-204-10f4d0c1-80ac-4e51-acf9-b918ef057a8d-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00098-3-ba66277d-68ee-4b82-86ac-0b7e1aab54f5-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00177-208-a545235b-7ccd-45f0-8d66-c3a6fd260072-0-00001.parquet

gs://gcs-buck

### 7. INSERT OVERWRITE Operation

In [23]:
#Creating a source table for overwriting 

overwrt_df = spark.sql(f"SELECT * FROM {FQTN} WHERE addr_state NOT IN ('IA','CA')")
overwrt_df.createOrReplaceTempView("overwrite_source_table")

In [24]:
#Overwrite Iceberg table with the values from above source table
spark.sql(f"INSERT OVERWRITE {FQTN} SELECT * FROM overwrite_source_table").show(truncate=False)

++
||
++
++



In [25]:
#
spark.sql(f"SELECT COUNT(*) as count  from {FQTN}").show(truncate=False)

+-----+
|count|
+-----+
|49   |
+-----+



In [26]:
#Fetching the latest metadata file

latest_metadata = !gsutil ls  $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/metadata/*.metadata.json | tail -1
LATEST_METADATA_FILE = latest_metadata[0]
print("LATEST_METADATA_FILE", LATEST_METADATA_FILE)

!gsutil cat {LATEST_METADATA_FILE} 

LATEST_METADATA_FILE gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/metadata/00004-3bd4d900-5e07-4acf-9935-0d557023d467.metadata.json
{
  "format-version" : 2,
  "table-uuid" : "9228ae9b-8c1d-4c94-bc1e-9205b86ffd30",
  "location" : "gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg",
  "last-sequence-number" : 5,
  "last-updated-ms" : 1715619153693,
  "last-column-id" : 2,
  "current-schema-id" : 0,
  "schemas" : [ {
    "type" : "struct",
    "schema-id" : 0,
    "fields" : [ {
      "id" : 1,
      "name" : "addr_state",
      "required" : false,
      "type" : "string"
    }, {
      "id" : 2,
      "name" : "loan_count",
      "required" : false,
      "type" : "long"
    } ]
  } ],
  "default-spec-id" : 0,
  "partition-specs" : [ {
    "spec-id" : 0,
    "fields" : [ ]
  } ],
  "last-partition-id" : 999,
  "default-sort-order-id" : 0,
  "sort-o

**NOTE:**
INSERT OVERWRITE operation performs an atomic overwrite on the data file.
Notice the operation summary of the current snapshot in above metadata file

In [27]:
# Note how the update created a new parquet file and added more metadata files in the metadata directory
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-525c32e9-c437-4d4c-a7a0-c6fac91bba85-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-204-10f4d0c1-80ac-4e51-acf9-b918ef057a8d-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-409-0e90780c-f8a8-40ff-a14d-6defc2157af7-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00098-3-ba66277d-68ee-4b82-86ac-0b7e1aab54f5-0-00001.parquet
gs://gcs-bucke

### 8. MERGE INTO Operation

In [28]:
#Create a Source Dataframe to merge
# IN exists - update operation
# CA, IA dont exist - insert operation
# Together - an upsert operation
toBeMergedRows = [('IA', 11111), ('CA', 11111), ('IN', 11111)]
toBeMergedColumns = ['addr_state', 'loan_count']
toBeMergedDF = spark.createDataFrame(toBeMergedRows, toBeMergedColumns)
toBeMergedDF.createOrReplaceTempView("to_be_merged_table")
toBeMergedDF.orderBy("addr_state").show(3)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|        CA|     11111|
|        IA|     11111|
|        IN|     11111|
+----------+----------+



In [29]:
#Iceberg Table before merging
spark.sql(f"SELECT addr_state,loan_count FROM {FQTN} WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|IN        |7511      |
+----------+----------+



In [30]:
#build merge statement
mergeSQLStatement = f"MERGE INTO {FQTN} as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * "

print(mergeSQLStatement)

MERGE INTO loan_db.loans_by_state_iceberg as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * 


In [31]:
spark.sql(mergeSQLStatement).show(truncate=False)

++
||
++
++



In [32]:
# Iceberg Table 
spark.sql(f"SELECT addr_state,loan_count FROM {FQTN} WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show(truncate=False)

+----------+----------+
|addr_state|loan_count|
+----------+----------+
|CA        |11111     |
|IA        |11111     |
|IN        |11111     |
+----------+----------+



In [33]:
# Merge created new set of files 
!gsutil ls -r $HIVE_METASTORE_WAREHOUSE_DIR/loan_db.db/{TABLE_NAME}/

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/:

gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/:
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-2-525c32e9-c437-4d4c-a7a0-c6fac91bba85-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-204-10f4d0c1-80ac-4e51-acf9-b918ef057a8d-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-409-0e90780c-f8a8-40ff-a14d-6defc2157af7-0-00001.parquet
gs://gcs-bucket-dll-hms-11002190840-e1664035-a2d9-4215-be46-45c40712/hive-warehouse/loan_db.db/loans_by_state_iceberg/data/00000-441-eb0fbb23-c499-40cf-9192-c8bc95b48657-0-00001.parquet
gs://gcs-buc

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK